# 뉴진스 RAG 만들기

https://python.langchain.com/docs/tutorials/rag/#retrieval-and-generation

# 데이터 불러오기

In [2]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://namu.wiki/w/NewJeans")
document = loader.load()
document

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://namu.wiki/w/NewJeans', 'title': 'NewJeans - 나무위키', 'language': 'ko'}, page_content='\n\n\nNewJeans - 나무위키\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n최근 변경최근 토론특수 기능NewJeans 최근 수정 시각: 2024-09-28 10:15:02798편집편집편집 권한이 부족합니다. 가입한지 15일 지난 사용자(이)여야 합니다. 해당 문서의 ACL 탭을 확인하시기 바랍니다.닫기토론역사분류NewJeans \xa0 ACL 규칙으로 일부 이용자의 편집이 제한된 문서입니다. \xa0 NewJeans 멤버들의 9월 11일자 라이브 방송 등 각종 이슈에 대한 내용은 민희진-HYBE 간 ADOR 경영권 분쟁/전개/2024년 9월 문서를의 번 문단을의  부분을, 에 대한 내용은  문서를의 번 문단을의 번 문단을의  부분을의  부분을, 에 대한 내용은  문서를의 번 문단을의 번 문단을의  부분을의  부분을, 에 대한 내용은  문서를의 번 문단을의 번 문단을의  부분을의  부분을, 에 대한 내용은  문서를의 번 문단을의 번 문단을의  부분을의  부분을, 에 대한 내용은  문서를의 번 문단을의 번 문단을의  부분을의  부분을, 에 대한 내용은  문서를의 번 문단을의 번 문단을의  부분을의  부분을, 에 대한 내용은  문서를의 번 문단을의 번 문단을의  부분을의  부분을, 에 대한 내용은  문서를의 번 문단을의 번 문단을의  부분을의  부분을, 에 대한 내용은  문서를의 번 문단을의 번 문단을의  부분을의  부분을 참고하십시오. \xa0 은(는) 여기로 연결됩니다. 본 그룹의 데뷔 음반에 대한 내용은 New Jeans(음반) 문서를의 번 문단을의  부분을, 본 그룹이 발매한 동명인 노래에 대한 내용은 New Jeans(노래) 문서를의 번 문단을의 번 문단을의  부분을의  부분을, 에 대한 내용은 

In [19]:
len(document)

1

In [20]:
type(document)

list

In [23]:
document[0].metadata

{'source': 'https://namu.wiki/w/NewJeans',
 'title': 'NewJeans - 나무위키',
 'language': 'ko'}

# 데이터 쪼개기

In [24]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(document)

len(all_splits)

46

In [26]:
len(all_splits[0].page_content)

15

In [27]:
all_splits[0].page_content

'NewJeans - 나무위키'

In [29]:
len(all_splits[1].page_content)

998

# 인덱싱

In [31]:
!pip install langchain-chroma

   ---------------------------------------- 0.0/604.0 kB ? eta -:--:--
   --------------------------- ----------- 419.8/604.0 kB 12.8 MB/s eta 0:00:01
   --------------------------------------- 604.0/604.0 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: chromadb
    Found existing installation: chromadb 0.5.5
    Uninstalling chromadb-0.5.5:
      Successfully uninstalled chromadb-0.5.5



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [45]:
from langchain_chroma import Chroma
from langchain.embeddings import OllamaEmbeddings

persist_directory = "./Newjeans_db"
embedding_model = OllamaEmbeddings(model='llama3.1')

vectorstore = Chroma.from_documents(
    documents=all_splits,
    embedding=embedding_model,
    persist_directory="./Newjeans_db"
)

# 검색 및 생성

https://python.langchain.com/docs/how_to/vectorstore_retriever/

In [46]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("뉴진스는 몇 인조야?")

retrieved_docs

[Document(metadata={'language': 'ko', 'source': 'https://namu.wiki/w/NewJeans', 'start_index': 30433, 'title': 'NewJeans - 나무위키'}, page_content='마차에서 5명이 내리는 장면이 나온다.[51]2021년 7월 9일, 방탄소년단의 Permission to Dance MV에 멤버 민지, 하니 두 명이 출연했었고, 민지는 2017년부터 연습생이었어서 2019년 플러스 글로벌 오디션 홍보 영상에도 출연했으며, 해당 오디션 및 캐스팅을 통해 선발되어 멤버 전원이 10대인 소녀들을 중심으로 기획되었다.민희진은 데뷔팀 지향점을 \'숙련\'보다는 \'즐기기\'에 두고 있고, 진심으로 즐기는 사람에게서 나오는 에너지는 엄청나게 강력해서 보는 사람까지도 춤추게 만든다고 한다고 말했다. 그리고 업계에서 (내가 아닌) 남이 말해줬을 때 듣기 좋았던 말 중 가장 기분 좋은 두 단어가 \'세계관\'과 \'아티스트\'라고 말했는데, 대신 인위적으로 만들어낸 설정보다 자연스러운 흐름과 복선을 좋아한다고 밝혔다.2021년 12월 1일, 민희진이 유 퀴즈 온 더 블럭에 출연해 HYBE에 입사한 2019년부터 너무 잘하는 프로듀서와 이미 타이틀 곡을 준비해놨다고 하고, "분명히 굉장히 새로운 반향을 불러일으키지 않을까 심히 기대하고 있다"라고 말하며 2022년에 데뷔 시킬 것이라고 밝혔다.[52]민희진 대표는 멤버들 부모님들과 나이가 비슷해서 너무 신기했고, 좋은 이모가 되고 싶고 좋은 엄마가 돼주고 싶다며 한 명씩 주말마다 집에 불러서 요리도 해주고 산책도 하며 친해지는 시간을 가졌으며, 멤버들이 자연스럽게 아티스트가 되는 과정을 만들어주고 싶었다고 한다.멤버들이 미숙한데 진짜 노력을 많이 해서 막 병아리 같고 예뻐보였다고 한다. 그래서 \'즐겁게 해보자, 그러면 굉장히 다른 바이브가 나올거다\'라고 말했다고 한다.2022년 1월 7일, 다른 인물을 통해 New Jeans라는 상표가

# 체인 만들기

In [47]:
# 모델
from langchain.llms import Ollama

llm = Ollama(model="llama3.1")

In [48]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate

rag_prompt = PromptTemplate.from_template(
    """
    Given the following context:

    {context}

    Answer the following question:

    {question}

    Answer:
    """
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("뉴진스는 몇 인조야?"):
    print(chunk, end="", flush=True)

5인조(New)야